In [1]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp as OR

In [2]:
graph = pd.read_csv('mis_seat_packing_example.csv',index_col=0).astype(int)
graph.columns = graph.columns.astype(int)

In [3]:
def mis(graph, integer=False):
    """A model for solving the maximum independent set problem.
    
    Args:
        graph (pd.DataFrame): Graph represented by an adjacency matrix.
    """
    NODES = list(graph.index)                # nodes
    EDGES = []                               # edges
    for i in NODES:
        for j in NODES:
            if i <= j and graph.at[i,j] == 1:
                EDGES.append((i,j))
    
    # define model
    m = OR.Solver('mis', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {} 
    for i in NODES:
        if integer:
            x[i] = m.IntVar(0, 1, ('%s' % (i)))
        else:
            x[i] = m.NumVar(0, 1, ('%s' % (i)))
    
    # objective function
    m.Maximize(sum(x[i] for i in NODES))
        
    # subject to: no vertices in the set share an edge
    for i,j in EDGES:
        m.Add(x[i] + x[j] <= 1)

    return m,x

In [4]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [5]:
m,x = mis(graph)
solve(m)

Solution:
Objective value = 6.0
1 : 0.5
2 : 0.5
3 : 0.5
4 : 0.5
5 : 0.5
6 : 0.5
7 : 0.5
8 : 0.5
9 : 0.5
10 : 0.5
11 : 0.5
12 : 0.5


In [6]:
m,x = mis(graph, integer=True)
solve(m)

Solution:
Objective value = 4.0
1 : 1.0
2 : 0.0
3 : 0.0
4 : 1.0
5 : 0.0
6 : 0.0
7 : 0.0
8 : 0.0
9 : 1.0
10 : 0.0
11 : 0.0
12 : 1.0
